In [1]:
import os
import dask.dataframe as dd
import datetime
import logging
import numpy as np
import pandas as pd
import time  # Para medir o tempo de execução
from dask.distributed import Client

# Configurar o logging para visualizar os logs de informação
logging.basicConfig(level=logging.INFO)

# Configurar o cliente Dask
client = Client(n_workers=10, threads_per_worker=1, memory_limit='6.4GB')
logging.info(client)

raw_dataset_path = '../datasets/BTCUSDT-Trades-compressed/'
output_base_path = '../output'
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
output_path = f'{output_base_path}_v{timestamp}'

def read_parquet_files_optimized(raw_dataset_path, file):
    parquet_pattern = os.path.join(raw_dataset_path, file)
    df_dask = dd.read_parquet(
        parquet_pattern,
        columns=['price', 'qty', 'quoteQty', 'time'],
        engine='pyarrow',
        dtype={'price': 'float32', 'qty': 'float32', 'quoteQty': 'float32'}
    )
    return df_dask

def assign_side_optimized(df):
    df['side'] = np.where(df['price'].shift() > df['price'], 1,
                          np.where(df['price'].shift() < df['price'], -1, np.nan))
    # Preencher NaNs após ffill() com um valor padrão, por exemplo, 1
    df['side'] = df['side'].ffill().fillna(1).astype('int8')
    return df

def apply_operations_optimized(df_dask, meta):
    df_dask = df_dask.map_partitions(assign_side_optimized, meta=meta)
    df_dask['dollar_imbalance'] = df_dask['quoteQty'] * df_dask['side']
    return df_dask

def create_imbalance_dollar_bars(partition, init_T, init_dif, res_init, alpha_volume, alpha_imbalance):
    """
    Função original para criar barras de desequilíbrio em dólares.
    """
    exp_T     = init_T
    exp_dif   = init_dif
    threshold = exp_T * init_dif

    bars = []

    # Variáveis de agregação de uma barra
    if len(res_init) > 0:
        bar_open = res_init[0]
        bar_high = res_init[1]
        bar_low = res_init[2]
        bar_close = res_init[3]
        bar_start_time = res_init[4]
        bar_end_time = res_init[5]
        current_imbalance = res_init[6]
        buy_volume_usd = res_init[7]
        total_volume_usd = res_init[8]
        total_volume = res_init[9]
    else:
        bar_open = None
        bar_high = -float('inf')
        bar_low = float('inf')
        bar_close = None
        bar_start_time = None
        bar_end_time = None
        current_imbalance = 0
        buy_volume_usd = 0
        total_volume_usd = 0
        total_volume = 0

    price_col = 'price'
    time_col = 'time'
    imbalance_col = 'dollar_imbalance'
    volume_col = 'qty'

    try:
        # Convertendo a partição para arrays numpy
        prices = partition['price'].values
        times = partition['time'].values
        imbalances = partition['dollar_imbalance'].values
        sides = partition['side'].values
        qtys = partition['qty'].values

        for i in range(len(prices)):
            if bar_open is None:
                bar_open = prices[i]
                bar_start_time = times[i]

            # Atualiza valores de OHLC
            trade_price = prices[i]
            bar_high = max(bar_high, trade_price)
            bar_low = min(bar_low, trade_price)
            bar_close = trade_price

            # Soma o volume (ou outra métrica de desequilíbrio)
            trade_imbalance = imbalances[i]

            if sides[i] > 0:
                buy_volume_usd += trade_imbalance

            total_volume += qtys[i]
            total_volume_usd += abs(trade_imbalance)
            current_imbalance += trade_imbalance
            imbalance = abs(current_imbalance)

            # Verifica se a soma já ultrapassou o threshold
            if imbalance >= threshold:
                bar_end_time = times[i]

                # Salvar a barra formada
                bars.append({
                    'start_time': bar_start_time,
                    'end_time': bar_end_time,
                    'open': bar_open,
                    'high': bar_high,
                    'low': bar_low,
                    'close': bar_close,
                    'imbalance_col': current_imbalance,
                    'total_volume_buy_usd': buy_volume_usd,
                    'total_volume_usd': total_volume_usd,
                    'total_volume': total_volume
                })

                # Exponential-weighted updates
                if exp_dif == 1:
                    exp_T   = total_volume_usd
                    exp_dif = abs(2 * buy_volume_usd / total_volume_usd - 1)
                else:
                    exp_T   += alpha_volume * (total_volume_usd   - exp_T)
                    exp_dif += alpha_imbalance * (abs(2 * buy_volume_usd / total_volume_usd - 1) - exp_dif)
                # Reset accumulators
                threshold = exp_T * abs(exp_dif)

                # Variáveis de agregação de uma barra
                bar_open = None
                bar_high = -float('inf')
                bar_low = float('inf')
                bar_close = None
                bar_start_time = None
                bar_end_time = None
                current_imbalance = 0
                buy_volume_usd = 0
                total_volume_usd = 0
                total_volume = 0
    finally:
        if current_imbalance == 0:
            res_init = []
        else:
            res_init = [bar_open, bar_high, bar_low, bar_close, bar_start_time,
                       bar_end_time, current_imbalance, buy_volume_usd, total_volume_usd, total_volume]

    return bars, exp_T, exp_dif, res_init

def batch_create_imbalance_dollar_bars_optimized(df_dask, init_T, init_dif, res_init, alpha_volume, alpha_imbalance):
    results = []
    start_time = time.time()  # Inicia a contagem do tempo total

    # Variáveis para cálculo de tempo estimado
    total_time_spent = 0  # Tempo total gasto até o momento
    completed_batches = 0  # Contador de batches processados
    estimated_time_left = 0  # Estimativa de tempo restante

    for partition in range(df_dask.npartitions):
        part_start_time = time.time()  # Inicia o tempo de execução para o batch

        logging.info(f"Processando partição {partition + 1} de {df_dask.npartitions}")

        part = df_dask.get_partition(partition).compute()
        bars, init_T, init_dif, res_init = create_imbalance_dollar_bars(
            part, init_T, init_dif, res_init, alpha_volume, alpha_imbalance
        )
        results.append(pd.DataFrame(bars))

        part_end_time = time.time()  # Fim do tempo de execução para o batch
        batch_time = part_end_time - part_start_time
        total_time_spent += batch_time
        completed_batches += 1

        # Calcula o tempo médio por batch
        average_time_per_batch = total_time_spent / completed_batches

        # Calcula o número de batches restantes
        remaining_batches = df_dask.npartitions - completed_batches

        # Estima o tempo restante
        estimated_time_left = average_time_per_batch * remaining_batches

        # Exibe informações de progresso
        logging.info(f"Tempo médio por batch: {average_time_per_batch:.2f} segundos")
        logging.info(f"Batches restantes: {remaining_batches}")
        logging.info(f"Tempo estimado restante: {estimated_time_left / 60:.2f} minutos")  # Tempo restante em minutos

    results_df = pd.concat(results, ignore_index=True)
    end_time = time.time()  # Fim do tempo total
    logging.info(f"Processamento completo em {total_time_spent / 60:.2f} minutos.")
    return results_df, init_T, init_dif, res_init

# Definir o meta DataFrame para map_partitions
meta = pd.DataFrame({
    'price': pd.Series(dtype='float32'),
    'qty': pd.Series(dtype='float32'),
    'quoteQty': pd.Series(dtype='float32'),
    'time': pd.Series(dtype='datetime64[ns]'),
    'side': pd.Series(dtype='int8')  # Adicionando a coluna 'side' como int8
})

# Listar arquivos
files = [f for f in os.listdir(raw_dataset_path) if os.path.isfile(os.path.join(raw_dataset_path, f))]
file_count = len(files)

results = pd.DataFrame()

init_T = 10_000
init_dif = 1
alpha_volume = 0.1
alpha_imbalance = 0.9
res = []
# file_count + 1
for number in range(1, 2):  # Ajuste para incluir o último arquivo
    logging.info(f"Dask processando arquivo {number} de {file_count}")
    file = f'combined_file_{number}.parquet'

    # Verifique se o arquivo existe para evitar erros
    if not os.path.exists(os.path.join(raw_dataset_path, file)):
        logging.warning(f"Arquivo {file} não encontrado. Pulando para o próximo.")
        continue

    df_dask = read_parquet_files_optimized(raw_dataset_path, file)
    df_dask = apply_operations_optimized(df_dask, meta)
    bars, init_T, init_dif, res = batch_create_imbalance_dollar_bars_optimized(
        df_dask, init_T, init_dif, res, alpha_volume, alpha_imbalance
    )
    results = pd.concat([results, bars], ignore_index=True)

output_file = f'{output_path}-{alpha_volume}-{alpha_imbalance}-{init_T}.xlsx'
results.to_excel(output_file, index=False)
logging.info(f"Resultados salvos em {output_file}")


/opt/anaconda3/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 56175 instead
  warnings.warn(
INFO:root:<Client: 'tcp://127.0.0.1:56176' processes=10 threads=10, memory=59.60 GiB>
INFO:root:Dask processando arquivo 1 de 12
INFO:root:Processando partição 1 de 43
INFO:root:Tempo médio por batch: 9.12 segundos
INFO:root:Batches restantes: 42
INFO:root:Tempo estimado restante: 6.38 minutos
INFO:root:Processando partição 2 de 43
INFO:root:Tempo médio por batch: 9.11 segundos
INFO:root:Batches restantes: 41
INFO:root:Tempo estimado restante: 6.23 minutos
INFO:root:Processando partição 3 de 43
INFO:root:Tempo médio por batch: 9.14 segundos
INFO:root:Batches restantes: 40
INFO:root:Tempo estimado restante: 6.09 minutos
INFO:root:Processando partição 4 de 43
INFO:root:Tempo médio por batch: 9.16 segundos
INFO:root:Batches restantes: 39
INFO:root:Tempo estimado restante: 5.

In [ ]:
        # Convertendo a partição para arrays numpy
        prices = partition['price'].values
        times = partition['time'].values
        imbalances = partition['dollar_imbalance'].values
        sides = partition['side'].values
        qtys = partition['qty'].values

        for i in range(len(prices)):
            if bar_open is None:
                bar_open = prices[i]
                bar_start_time = times[i]

            # Atualiza valores de OHLC
            trade_price = prices[i]
            bar_high = max(bar_high, trade_price)
            bar_low = min(bar_low, trade_price)
            bar_close = trade_price

            # Soma o volume (ou outra métrica de desequilíbrio)
            trade_imbalance = imbalances[i]

            if sides[i] > 0:
                buy_volume_usd += trade_imbalance

            total_volume += qtys[i]
            total_volume_usd += abs(trade_imbalance)
            current_imbalance += trade_imbalance
            imbalance = abs(current_imbalance)

            # Verifica se a soma já ultrapassou o threshold
            if imbalance >= threshold:
                bar_end_time = times[i]